In [1]:
import cv2
import numpy as np

In [2]:
TRANSFORMATIONS = [

    # Translation
    np.array([
        [1, 0, -100],
        [0, 1, -200],
        [0, 0, 1]
    ]),

    # Euclidean
    np.array([
        [0.866, -0.5, 0],
        [0.5, 0.866, 0],
        [0, 0, 1]
    ]),

    # Affine
    np.array([
        [1, 2, -800],
        [0, 2, -400],
        [0, 0, 1]
    ]),

    # Projection
    np.array([
        [1, 0, 100],
        [0, 1, 200],
        [0, 0, 1]
    ])

]

In [3]:
def Apply_Transformation(img, transform_idx):

    # Get transformation to apply
    # Take the inverse to perform inverse warping
    transform_array = TRANSFORMATIONS[transform_idx]

    # Initialize new image
    new_img = np.zeros(shape=img.shape)

    coor_pairs = []

    # Loop through coordinates and
    # apply inverse transformation
    for r in range(img.shape[0]):
        for c in range(img.shape[1]):
            current_coor = np.array([c, r, 1])
            new_coor = np.matmul(transform_array, current_coor)
            new_x = new_coor[0] / new_coor[2]
            new_y = new_coor[1] / new_coor[2]
            # Update new image if old coordinate
            # is within the image coordinate bounds
            if new_y < img.shape[0] and \
                new_x < img.shape[1] and \
                new_y > 0 and \
                new_x > 0:

                    if new_y % 1 == 0 and new_x % 1 == 0 and img[int(new_y), int(new_x), 0] == 255 and len(coor_pairs) < transform_idx + 1:
                        coor_pairs += [[(new_x, new_y), (c, r)]]

                    new_img[r, c] = img[int(new_y), int(new_x)]

    return new_img, coor_pairs

In [4]:
simple_image = np.zeros(shape=(1024,1024,3))

simple_image[300:500, 300] = 255
simple_image[300:500, 500] = 255
simple_image[300, 300:500] = 255
simple_image[500, 300:500] = 255

cv2.imwrite('part2-images/simple.jpg', simple_image)

True

In [5]:
translated_img, translated_coors = Apply_Transformation(simple_image, 0)
cv2.imwrite('part2-images/Simple_Translation_2.jpg', translated_img)

True

In [6]:
euclidean_img, euclidean_coors = Apply_Transformation(simple_image, 1)
cv2.imwrite('part2-images/Simple_Euclidean.jpg', euclidean_img)

True

In [7]:
affine_img, affine_img_coors = Apply_Transformation(simple_image, 2)
cv2.imwrite('part2-images/Simple_Affine.jpg', affine_img)

True

In [8]:
affine_img_coors

[[(300.0, 300.0), (400, 350)],
 [(301.0, 300.0), (401, 350)],
 [(302.0, 300.0), (402, 350)]]

In [9]:
def Solve_Transformation(coor_pairs):

    equation_array = np.array([])
    answer_array = np.array([])
    results_array = np.array([])

    if len(coor_pairs) == 1:

        equation_array = np.array([
            [coor_pairs[0][0][0], 0],
            [0, coor_pairs[0][0][1]]
        ])

        answer_array = np.array([coor_pairs[0][1][0], coor_pairs[0][1][1]])

        solved = np.linalg.solve(equation_array, answer_array)

        results_array = np.array([
            [solved[0], 0, 0],
            [0, solved[1], 0],
            [0, 0, 1]
        ])

    return results_array

    

In [10]:
found_transform = Solve_Transformation(translated_coors)

In [11]:
found_transform

array([[1.33333333, 0.        , 0.        ],
       [0.        , 1.66666667, 0.        ],
       [0.        , 0.        , 1.        ]])

In [12]:
def Apply_Transformation_Two(img, transform_array):

    # Initialize new image
    new_img = np.zeros(shape=img.shape)

    # Loop through coordinates and
    # apply inverse transformation
    for r in range(img.shape[0]):
        for c in range(img.shape[1]):
            current_coor = np.array([c, r, 1])
            new_coor = np.matmul(transform_array, current_coor)
            new_x = new_coor[0] / new_coor[2]
            new_y = new_coor[1] / new_coor[2]
            # Update new image if old coordinate
            # is within the image coordinate bounds
            if new_y < img.shape[0] and \
                new_x < img.shape[1] and \
                new_y > 0 and \
                new_x > 0:

                    new_img[r, c] = img[int(new_y), int(new_x)]

    return new_img

In [13]:
thingy = Apply_Transformation_Two(simple_image, found_transform)
cv2.imwrite('Simple_Translation_2.jpg', thingy)

True

In [14]:
[300, 300, 1]

[300, 300, 1]

In [17]:
import numpy as np
one = np.array([
    [300, 300, 1, 0, 0, 0], 
    [0, 0, 0, 300, 300, 1],
    [301, 300, 1, 0, 0, 0], 
    [0, 0, 0, 301, 300, 1],
    [302, 300, 1, 0, 0, 0],  
    [0, 0, 0, 302, 300, 1],
])

two = np.array([400, 350, 401, 350, 402, 350])



np.linalg.solve(one, two)

LinAlgError: Singular matrix

In [ ]:
t = np.linalg.solve(one, two)

one_orig = np.array([
        300, 300, 1
    ])

solved = np.array([
    [t[0], t[1],  t[2]],
    [t[3], t[4],  t[5]],
    [0, 0, 1]
])

np.matmul(solved, one_orig)

array([400., 350.,   1.])

In [ ]:
# “book1.jpg”: {(318, 256), (534, 372), (316, 670), (73, 473)}
# “book2.jpg”: {(141, 131), (480, 159), (493, 630), (64, 601)}

In [11]:
import numpy as np
# one = np.array([
#     [318, 256, 1, 1, 0, 0, 0, 0], 
#     [0, 0, 0, 0, 318, 256, 1, 1],
#     [534, 372, 1, 1, 0, 0, 0, 0], 
#     [0, 0, 0, 0, 534, 372, 1, 1],
#     [316, 670, 1, 1, 0, 0, 0, 0], 
#     [0, 0, 0, 0, 316, 670, 1, 1],
#     [73, 473, 1, 1, 0, 0, 0, 0],  
#     [0, 0, 0, 0, 73, 473, 1, 1],
# ])

# two = np.array([141, 131, 480, 159, 493, 630, 64, 601])

# np.linalg.solve(one, two)

one = np.array([
    [318, 256, 1, 0, 0, 0, 0, 0, 0], 
    [0, 0, 0, 318, 256, 1, 0 , 0, 0],
    [0, 0, 0, 0, 0, 0, 318, 256, 1],
    [534, 372, 1, 0, 0, 0, 0, 0, 0], 
    [0, 0, 0, 534, 372, 1, 0, 0, 0], 
    [0, 0, 0, 0, 0, 0, 534, 372, 1],
    [316, 670, 1, 0, 0, 0, 0, 0, 0], 
    [0, 0, 0, 316, 670, 1, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 316, 670, 1],
    [73, 473, 1, 0, 0, 0, 0, 0, 0], 
    [0, 0, 0, 73, 473, 1, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 73, 473, 1]
])

two = np.array([141, 131, 1, 480, 159, 1, 493, 630, 1, 64, 601, 1])

#np.linalg.solve(one, two)
one_pinv = np.linalg.pinv(one)
print(one.shape, one_pinv.shape)
np.dot(one_pinv, two)


(12, 9) (9, 12)


array([ 1.08879383e+00,  8.53339399e-01, -4.21114305e+02, -7.04562021e-01,
        1.18267744e+00,  7.52099293e+01, -8.73660953e-14,  6.10470875e-14,
        1.00000000e+00])

In [5]:
#t = np.linalg.solve(one, two)
t = np.dot(one_pinv, two)

one_orig = np.array([
        [318, 534, 316, 73],
        [256, 372, 670, 473],
        [1, 1, 1, 1]
    ])

# (64, 601)

solved = np.array([
    [t[0], t[1],  t[2]],
    [t[3], t[4],  t[5]],
    [t[6], t[7], t[8]]
])

out = np.matmul(solved, one_orig)

In [6]:
out

array([[143.57701939, 477.74385706, 494.68194299,  61.99718057],
       [153.92463188, 138.92981864, 644.96221726, 583.18333221],
       [  1.        ,   1.        ,   1.        ,   1.        ]])

In [89]:
out[0] / out[2]

0.0018131077965894726

In [13]:
simple_image = np.zeros(shape=(1024,1024,3))

simple_image[700:900, 700] = 255
simple_image[700:900, 900] = 255
simple_image[700, 700:900] = 255
simple_image[900, 700:900] = 255

cv2.imwrite('part2-images/simple_img2.jpg', simple_image)

True

In [17]:
first_corners = np.array([
    [300, 300, 500, 500],
    [300, 500, 300, 500],
    [1, 1, 1]
])

second_corners = np.array([
    [700, 700, 900, 900],
    [700, 900, 700, 900],
    [1, 1, 1]
])

In [ ]:
first_corners = np.array([
    [300, 300, 1],
    [0, 300, 1],
    [0, 0, 1]
])

second_corners = np.array([700, 700, 1])

In [116]:
first_corners = np.array([
    [300, 0, 1],
    [0, 300, 1],
    [0, 0, 1]
])

second_corners = np.array([700, 700, 1])

In [120]:
ans = np.linalg.solve(first_corners, second_corners)

C:\Ed\Anaconda\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  """Entry point for launching an IPython kernel.


In [119]:
ans

array([2.33, 2.33, 1.  ])

In [115]:
np.matmul(first_corners[0], ans[0])

ValueError: matmul: Input operand 1 does not have enough dimensions (has 0, gufunc core with signature (n?,k),(k,m?)->(n?,m?) requires 1)

In [89]:
first_corners[1]

array([  0, 500,   1])

In [90]:
ans[0]

array([2.33329541, 1.79997724, 0.01137761])

In [ ]:
# Translation
input_matrix = np.array([
    [300, 300, 1]
])

output_matrix = np.array([
    [700, 700, 1]
])

# Euclidean
input_matrix = np.array([
    [300, 300, 1],
    [300, 500, 1]
])

output_matrix = np.array([
    [700, 700, 1],
    [800, 950, 1]
])

# Affine
input_matrix = np.array([
    [300, 300, 1],
    [300, 500, 1],
    [500, 500, 1]
])

output_matrix = np.array([
    [700, 700, 1],
    [800, 875, 1],
    []
])